In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
#loading the itents file

with open('Intent1.json') as file:
    data = json.load(file)
    
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for text in intent['text']:
        training_sentences.append(text)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])
    
    if intent['intent'] not in labels:
        labels.append(intent['intent'])
        
num_classes = len(labels)

In [3]:
training_labels

['Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'Greeting',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'GreetingResponse',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreeting',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CourtesyGreetingResponse',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'CurrentHumanQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'NameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery',
 'RealNameQuery'

In [4]:
training_sentences[:5]

['Hi', 'Hi there', 'Hola', 'Hello', 'Hello there']

In [5]:
labels

['Greeting',
 'GreetingResponse',
 'CourtesyGreeting',
 'CourtesyGreetingResponse',
 'CurrentHumanQuery',
 'NameQuery',
 'RealNameQuery',
 'TimeQuery',
 'Thanks',
 'NotTalking2U',
 'UnderstandQuery',
 'Shutup',
 'Swearing',
 'GoodBye',
 'CourtesyGoodBye',
 'WhoAmI',
 'Clever',
 'Gossip',
 'Jokes',
 'PodBayDoor',
 'PodBayDoorResponse',
 'SelfAware']

In [6]:
num_classes = len(labels)
num_classes

22

In [7]:
#encoding the training labels
encoder = LabelEncoder()
training_labels = encoder.fit_transform(training_labels)

In [8]:
training_labels[:5]

array([7, 7, 7, 7, 7], dtype=int64)

In [9]:
vocab_size =1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [10]:
padded_sequences[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 51])

In [11]:
word_index

{'<OOV>': 1,
 'you': 2,
 'is': 3,
 'me': 4,
 'are': 5,
 'i': 6,
 'what': 7,
 'the': 8,
 'thanks': 9,
 'do': 10,
 'can': 11,
 'name': 12,
 'am': 13,
 'please': 14,
 'your': 15,
 'tell': 16,
 'a': 17,
 'pod': 18,
 'bay': 19,
 'door': 20,
 'adam': 21,
 'bella': 22,
 'open': 23,
 'to': 24,
 'real': 25,
 'time': 26,
 'not': 27,
 'prove': 28,
 'my': 29,
 'it': 30,
 'how': 31,
 'talking': 32,
 'gossip': 33,
 'why': 34,
 'good': 35,
 'call': 36,
 'know': 37,
 'see': 38,
 'hello': 39,
 'user': 40,
 'this': 41,
 'great': 42,
 'who': 43,
 'thank': 44,
 "wasn't": 45,
 'bye': 46,
 'goodbye': 47,
 'very': 48,
 'self': 49,
 'aware': 50,
 'hi': 51,
 'there': 52,
 'doing': 53,
 'well': 54,
 'ok': 55,
 'for': 56,
 'quiet': 57,
 'clever': 58,
 'girl': 59,
 'any': 60,
 'some': 61,
 'joke': 62,
 'will': 63,
 'have': 64,
 'conscious': 65,
 'hola': 66,
 'hya': 67,
 'hope': 68,
 'think': 69,
 "what's": 70,
 'understand': 71,
 'saying': 72,
 'be': 73,
 'up': 74,
 'fuck': 75,
 'anyone': 76,
 'in': 77,
 'camera'

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 50)                850       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 22)                1122      
                                                                 
Total params: 20,522
Trainable params: 20,522
Non-trainable params: 0
____________________________________________________

In [13]:
history= model.fit(padded_sequences, np.array(training_labels), epochs=500)

Epoch 1/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0918 - accuracy: 0.0559
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0896 - accuracy: 0.0559
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0888 - accuracy: 0.0559
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0880 - accuracy: 0.0559
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0874 - accuracy: 0.0559
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0863 - accuracy: 0.0559
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0854 - accuracy: 0.0559
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0842 - accuracy: 0.0559
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0829 - accuracy: 0.0559
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0816 - accuracy: 0.0559
Epoch 11/500
5/5 [===========

5/5 [==============================] - 0s 2ms/step - loss: 0.9818 - accuracy: 0.8252
Epoch 85/500
5/5 [==============================] - 0s 2ms/step - loss: 0.9624 - accuracy: 0.8112
Epoch 86/500
5/5 [==============================] - 0s 2ms/step - loss: 0.9418 - accuracy: 0.8042
Epoch 87/500
5/5 [==============================] - 0s 2ms/step - loss: 0.9312 - accuracy: 0.7902
Epoch 88/500
5/5 [==============================] - 0s 2ms/step - loss: 0.9032 - accuracy: 0.8042
Epoch 89/500
5/5 [==============================] - 0s 2ms/step - loss: 0.8863 - accuracy: 0.7972
Epoch 90/500
5/5 [==============================] - 0s 2ms/step - loss: 0.8676 - accuracy: 0.8042
Epoch 91/500
5/5 [==============================] - 0s 2ms/step - loss: 0.8524 - accuracy: 0.8531
Epoch 92/500
5/5 [==============================] - 0s 2ms/step - loss: 0.8330 - accuracy: 0.8601
Epoch 93/500
5/5 [==============================] - 0s 2ms/step - loss: 0.8165 - accuracy: 0.8601
Epoch 94/500
5/5 [===============

5/5 [==============================] - 0s 2ms/step - loss: 0.1753 - accuracy: 0.9930
Epoch 167/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1718 - accuracy: 0.9930
Epoch 168/500
5/5 [==============================] - 0s 1ms/step - loss: 0.1716 - accuracy: 0.9930
Epoch 169/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1646 - accuracy: 0.9930
Epoch 170/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1612 - accuracy: 0.9930
Epoch 171/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1581 - accuracy: 0.9930
Epoch 172/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1530 - accuracy: 0.9930
Epoch 173/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1493 - accuracy: 0.9930
Epoch 174/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1466 - accuracy: 0.9930
Epoch 175/500
5/5 [==============================] - 0s 2ms/step - loss: 0.1432 - accuracy: 0.9930
Epoch 176/500
5/5 [=====

5/5 [==============================] - 0s 2ms/step - loss: 0.0358 - accuracy: 1.0000
Epoch 249/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0352 - accuracy: 1.0000
Epoch 250/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0346 - accuracy: 1.0000
Epoch 251/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0340 - accuracy: 1.0000
Epoch 252/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0335 - accuracy: 1.0000
Epoch 253/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 254/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0326 - accuracy: 1.0000
Epoch 255/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0321 - accuracy: 1.0000
Epoch 256/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0318 - accuracy: 1.0000
Epoch 257/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0312 - accuracy: 1.0000
Epoch 258/500
5/5 [=====

5/5 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 331/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 332/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 333/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 334/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 335/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0119 - accuracy: 1.0000
Epoch 336/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 337/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0117 - accuracy: 1.0000
Epoch 338/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 339/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 340/500
5/5 [=====

5/5 [==============================] - 0s 2ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 413/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 414/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 415/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 416/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 417/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 418/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 419/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 420/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 421/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 422/500
5/5 [=====

5/5 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 495/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 496/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 497/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 498/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 499/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 500/500
5/5 [==============================] - 0s 1ms/step - loss: 0.0032 - accuracy: 1.0000


In [14]:
import pickle

model.save("chat_model2")

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)    

INFO:tensorflow:Assets written to: chat_model2\assets
